In [1]:
import sys;sys.path.append('..')
from ppanlp import *
ppa = PPA()
pd.options.display.max_rows=100

2023-12-06 21:33:46,683 | booting PPACorpus at /home/.../ppa_data/corpus
2023-12-06 21:33:46,684 | | reading metadata
2023-12-06 21:33:46,731 | | 0.05 seconds
2023-12-06 21:33:47,048 | 0.36 seconds


In [3]:
qd=dict(min_doc_len=25,max_per_cluster=50,frac=1)
tm = ppa.topic_model(**qd)
tm.mdl

2023-12-06 21:33:57,236 | loading or generating model
2023-12-06 21:33:57,237 | | importing BERTopic
2023-12-06 21:34:01,368 | | 4.13 seconds
2023-12-06 21:34:01,369 | | loading model from disk: /home/.../model.bin
2023-12-06 21:34:07,620 | | 6.25 seconds
2023-12-06 21:34:07,621 | 10.38 seconds


In [8]:
# tm.page2topic

In [9]:
ner = ppa.ner_model()

In [23]:
@cache
def person_cooccurence(self, min_page_count=25, lim=None, topic_model = None, funcs = [odds_ratio, fisher_exact], **kwargs):
    data = self.persons_anno_pagedata
    data = (
        random.sample(data,lim) 
        if lim and len(data)>lim 
        else data
    )
    toppref='TOPIC_'
    aupref='AUTHOR_'
    prpref='PERIOD_'
    person1 = Counter()
    person2 = Counter()
    pair_pages = defaultdict(set)
    numpages=0
    allppl=set()
    for pagedata in data:
        pageid=pagedata[0]['page_id']
        if topic_model and not pageid in topic_model.page2topic: continue
        numpages+=1
        workid='.'.join(pageid.split('.')[:-1])
        text = self.corpus.textd[workid]
        pageppl = {d['ent'] for d in pagedata}

        # if text.author: 
            # pageppl.add(aupref+text.author)

        if text.year>0:
            prd=50
            prd1=text.year//prd*prd
            prd2=prd1+prd
            pageppl.add(f'{prpref}{prd1}-{prd2}')

        if topic_model:
            topic=topic_model.page2topic.get(pageid)
            if topic and not topic.startswith('-1'):
                pageppl.add(toppref+topic)

        for x in pageppl: 
            person1[x]+=1
            allppl.add(x)
            for y in pageppl:
                if x<y:
                    person2[x,y]+=1

    def count_ind(x):
        return person1[x]
    def count_solo(x,y):
        return person1[x] - person2[x,y] - person2[y,x]
    def count_together(x,y):
        return person2[x,y]
    def count_neither(x,y):
        return numpages - count_together(x,y) - count_solo(x,y) - count_solo(y,x)

    person1sum=sum(person1.values())
    def prob_ind(x):
        return person1[x]/numpages
    def prob_obs(x,y):
        return person2[x,y]/numpages
    def prob_exp(x,y):
        return prob_ind(x) * prob_ind(y)
    def prob_obsexp(x,y):
        return prob_obs(x,y) / prob_exp(x,y)

    def gettype(x):
        if x.startswith(toppref): return 'Topic'
        if x.startswith(aupref): return 'Author'
        if x.startswith(prpref): return 'Period'
        return 'Person'

    def get_contingency_table(x,y):
        tl=count_together(x,y)
        tr=count_solo(x,y)
        bl=count_solo(y,x)
        br=count_neither(x,y)
        return ((tl,tr),(bl,br))

    def iter_res():
        minc=min_page_count
        cmps = [
            (x,y) 
            for x in allppl 
            for y in allppl 
            if x<y 
            and count_ind(x)>=minc 
            and count_ind(y)>=minc
            and not (x.startswith(aupref) and y.startswith(prpref))
            and not x.lower() in y.lower()
            and not y.lower() in x.lower()
        ]
        for x,y in tqdm(cmps,desc='Iterating over statistical comparisons',position=0):
            val_d={
                'entity_x':x, 
                'entity_y':y, 
                'type_x':gettype(x),
                'type_y':gettype(y),
                'num_total_x':count_ind(x), 
                'num_total_y':count_ind(y), 
                'num_solo_x':count_solo(x,y), 
                'num_solo_y':count_solo(y,x),
                'num_both_xy':count_together(x,y), 
                'num_neither_xy':count_neither(x,y), 
                'prob_x':prob_ind(x)*100,
                'prob_y':prob_ind(y)*100,
                'prob_xy_obs':prob_obs(x,y)*100,
                'prob_xy_exp':prob_exp(x,y)*100,
                'prob_xy_obsexp':prob_obsexp(x,y)*100,
            }
            ctbl=get_contingency_table(x,y)
            for func in funcs:
                res = func(ctbl)
                method=func.__name__
                stat=res.statistic if hasattr(res,'statistic') else None
                pval=res.pvalue if hasattr(res,'pvalue') else None
                if stat is not None: val_d[f'{method}'] = stat
                if pval is not None: val_d[f'{method}_p'] = pval
            if val_d.get('fisher_exact_p',1)!=1: 
                yield val_d
    o=list(iter_res())
    if not o: return pd.DataFrame()
    odf = pd.DataFrame(o).query('fisher_exact_p!=1').sort_values('odds_ratio',ascending=False)
    odf['prob_xy_obsexp_log']=odf['prob_xy_obsexp'].apply(np.log10)
    return odf




In [26]:
df = person_cooccurence(ner, lim=None, min_page_count=25, topic_model=tm)
df

Iterating over statistical comparisons: 100%|██████████| 171225/171225 [02:25<00:00, 1177.38it/s]


,entity_x,entity_y,type_x,type_y,num_total_x,num_total_y,num_solo_x,num_solo_y,num_both_xy,num_neither_xy,prob_x,prob_y,prob_xy_obs,prob_xy_exp,prob_xy_obsexp,odds_ratio,fisher_exact,fisher_exact_p,prob_xy_obsexp_log
24978,PERIOD_1850-1900,TOPIC_687_poe_poes_poets_edgar,Period,Topic,26329,26,26303,0,26,47591,35.618236,0.035173,0.035173,0.012528,280.755061,inf,inf,2.187911e-12,2.448328
53791,PERIOD_1900-1950,TOPIC_565_blake_blakes_blakean_doctrines,Period,Topic,24501,32,24469,0,32,49419,33.145292,0.043290,0.043290,0.014349,301.701971,inf,inf,4.442405e-16,2.479578
25198,PERIOD_1850-1900,TOPIC_299_quincey_quinceys_quixcey_literary,Period,Topic,26329,43,26286,0,43,47591,35.618236,0.058171,0.058171,0.020719,280.755061,inf,inf,5.156047e-20,2.448328
19957,PERIOD_1750-1800,TOPIC_213_suld_gud_yair_wallace,Period,Topic,11669,58,11611,0,58,62251,15.785985,0.078463,0.078463,0.012386,633.473305,inf,inf,2.804461e-47,2.801728
34587,Curtius,TOPIC_520_ionians_ionian_aegean_greeks,Person,Topic,120,25,97,2,23,73798,0.162338,0.033820,0.031115,0.000055,56672.000000,8668.489519,8749.247423,2.178918e-63,4.753369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42994,Collier,TOPIC_12_iliad_homeric_homer_ulysses,Person,Topic,358,601,358,601,0,72961,0.484307,0.813041,0.000000,0.003938,0.000000,0.000000,0.000000,1.273072e-01,-inf
42992,Collier,Mason,Person,Person,358,279,358,279,0,73283,0.484307,0.377435,0.000000,0.001828,0.000000,0.000000,0.000000,6.491557e-01,-inf
42984,Collier,Steele,Person,Person,358,390,358,390,0,73172,0.484307,0.527597,0.000000,0.002555,0.000000,0.000000,0.000000,2.722359e-01,-inf
42980,Collier,Sheridan,Person,Person,358,540,358,540,0,73022,0.484307,0.730519,0.000000,0.003538,0.000000,0.000000,0.000000,1.210752e-01,-inf


In [35]:
df=df.replace([np.inf, -np.inf], np.nan)
df=df[~df.odds_ratio.isna()]
df

,entity_x,entity_y,type_x,type_y,num_total_x,num_total_y,num_solo_x,num_solo_y,num_both_xy,num_neither_xy,prob_x,prob_y,prob_xy_obs,prob_xy_exp,prob_xy_obsexp,odds_ratio,fisher_exact,fisher_exact_p,prob_xy_obsexp_log
34587,Curtius,TOPIC_520_ionians_ionian_aegean_greeks,Person,Topic,120,25,97,2,23,73798,0.162338,0.033820,0.031115,0.000055,56672.000000,8668.489519,8749.247423,2.178918e-63,4.753369
18862,PERIOD_1550-1600,TOPIC_258_learninge_childe_learne_deede,Period,Topic,27,33,12,18,15,73875,0.036526,0.044643,0.020292,0.000016,124444.444444,5015.869189,5130.208333,2.190223e-45,5.094976
54505,De Quincey,TOPIC_299_quincey_quinceys_quixcey_literary,Person,Topic,346,43,306,3,40,73571,0.468074,0.058171,0.054113,0.000272,19873.638930,3196.080751,3205.708061,7.667223e-91,4.298277
2117,Collins,TOPIC_603_poets_poet_poetry_poem,Person,Topic,428,30,400,2,28,73490,0.579004,0.040584,0.037879,0.000235,16119.626168,2566.191468,2572.150000,3.973753e-61,4.207355
42390,Juliet,Romeo,Person,Person,349,296,115,62,234,73509,0.472132,0.400433,0.316558,0.001891,16744.056377,2400.908560,2412.497335,0.000000e+00,4.223861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42994,Collier,TOPIC_12_iliad_homeric_homer_ulysses,Person,Topic,358,601,358,601,0,72961,0.484307,0.813041,0.000000,0.003938,0.000000,0.000000,0.000000,1.273072e-01,NaN
42992,Collier,Mason,Person,Person,358,279,358,279,0,73283,0.484307,0.377435,0.000000,0.001828,0.000000,0.000000,0.000000,6.491557e-01,NaN
42984,Collier,Steele,Person,Person,358,390,358,390,0,73172,0.484307,0.527597,0.000000,0.002555,0.000000,0.000000,0.000000,2.722359e-01,NaN
42980,Collier,Sheridan,Person,Person,358,540,358,540,0,73022,0.484307,0.730519,0.000000,0.003538,0.000000,0.000000,0.000000,1.210752e-01,NaN


In [36]:
path=os.path.join(ppa.path_data,'data.ner+topic.stats.v2.csv')
df.to_csv(path,index=False)

In [31]:
df.type_x.value_counts()

Person    51295
Topic      2283
Period     1793
Name: type_x, dtype: int64

In [32]:
dfsig = df[df.fisher_exact_p<=.05]
# dfsig[dfsig.includes_topic]
dfsig_pos = dfsig[dfsig.odds_ratio>1]
# dfsig_pos.sample(100)
dfsig_pos.head(100)

,entity_x,entity_y,type_x,type_y,num_total_x,num_total_y,num_solo_x,num_solo_y,num_both_xy,num_neither_xy,prob_x,prob_y,prob_xy_obs,prob_xy_exp,prob_xy_obsexp,odds_ratio,fisher_exact,fisher_exact_p,prob_xy_obsexp_log
34587,Curtius,TOPIC_520_ionians_ionian_aegean_greeks,Person,Topic,120,25,97,2,23,73798,0.162338,0.033820,0.031115,0.000055,56672.000000,8668.489519,8749.247423,2.178918e-63,4.753369
18862,PERIOD_1550-1600,TOPIC_258_learninge_childe_learne_deede,Period,Topic,27,33,12,18,15,73875,0.036526,0.044643,0.020292,0.000016,124444.444444,5015.869189,5130.208333,2.190223e-45,5.094976
54505,De Quincey,TOPIC_299_quincey_quinceys_quixcey_literary,Person,Topic,346,43,306,3,40,73571,0.468074,0.058171,0.054113,0.000272,19873.638930,3196.080751,3205.708061,7.667223e-91,4.298277
2117,Collins,TOPIC_603_poets_poet_poetry_poem,Person,Topic,428,30,400,2,28,73490,0.579004,0.040584,0.037879,0.000235,16119.626168,2566.191468,2572.150000,3.973753e-61,4.207355
42390,Juliet,Romeo,Person,Person,349,296,115,62,234,73509,0.472132,0.400433,0.316558,0.001891,16744.056377,2400.908560,2412.497335,0.000000e+00,4.223861
30967,Gray,TOPIC_154_poet_poets_poems_poetry,Person,Topic,942,101,847,6,95,72972,1.274351,0.136634,0.128517,0.001741,7380.967396,1362.594765,1364.096812,9.349786e-174,3.868113
41052,Schipper,TOPIC_494_horn_verseform_verse_poem,Person,Topic,262,27,240,5,22,73653,0.354437,0.036526,0.029762,0.000129,22988.973707,1345.812606,1350.305000,3.959742e-50,4.361520
52363,Rowley,TOPIC_528_poems_poem_chatterton_chattertons,Person,Topic,146,32,126,12,20,73762,0.197511,0.043290,0.027056,0.000086,31643.835616,971.137487,975.687831,4.634554e-47,4.500289
22717,Cressida,Troilus,Person,Person,190,386,51,247,139,73483,0.257035,0.522186,0.188041,0.001342,14009.926370,808.759126,810.838851,1.561276e-283,4.146436
8102,Antony,Cleopatra,Person,Person,394,318,207,131,187,73395,0.533009,0.430195,0.252976,0.002293,11032.659707,505.091083,506.135081,0.000000e+00,4.042680


In [33]:
dfsig.type_x.value_counts()

Person    13325
Period     1097
Topic       505
Name: type_x, dtype: int64

In [34]:
dfsig

,entity_x,entity_y,type_x,type_y,num_total_x,num_total_y,num_solo_x,num_solo_y,num_both_xy,num_neither_xy,prob_x,prob_y,prob_xy_obs,prob_xy_exp,prob_xy_obsexp,odds_ratio,fisher_exact,fisher_exact_p,prob_xy_obsexp_log
34587,Curtius,TOPIC_520_ionians_ionian_aegean_greeks,Person,Topic,120,25,97,2,23,73798,0.162338,0.033820,0.031115,0.000055,56672.000000,8668.489519,8749.247423,2.178918e-63,4.753369
18862,PERIOD_1550-1600,TOPIC_258_learninge_childe_learne_deede,Period,Topic,27,33,12,18,15,73875,0.036526,0.044643,0.020292,0.000016,124444.444444,5015.869189,5130.208333,2.190223e-45,5.094976
54505,De Quincey,TOPIC_299_quincey_quinceys_quixcey_literary,Person,Topic,346,43,306,3,40,73571,0.468074,0.058171,0.054113,0.000272,19873.638930,3196.080751,3205.708061,7.667223e-91,4.298277
2117,Collins,TOPIC_603_poets_poet_poetry_poem,Person,Topic,428,30,400,2,28,73490,0.579004,0.040584,0.037879,0.000235,16119.626168,2566.191468,2572.150000,3.973753e-61,4.207355
42390,Juliet,Romeo,Person,Person,349,296,115,62,234,73509,0.472132,0.400433,0.316558,0.001891,16744.056377,2400.908560,2412.497335,0.000000e+00,4.223861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30783,Gray,TOPIC_12_iliad_homeric_homer_ulysses,Person,Topic,942,601,942,601,0,72377,1.274351,0.813041,0.000000,0.010361,0.000000,0.000000,0.000000,7.633517e-04,NaN
5277,Demosthenes,Jove,Person,Person,667,596,667,596,0,72657,0.902327,0.806277,0.000000,0.007275,0.000000,0.000000,0.000000,7.805955e-03,NaN
18419,Bede,Webster,Person,Person,375,1054,375,1054,0,72491,0.507305,1.425866,0.000000,0.007233,0.000000,0.000000,0.000000,7.718170e-03,NaN
42996,Collier,Keats,Person,Person,358,892,358,892,0,72670,0.484307,1.206710,0.000000,0.005844,0.000000,0.000000,0.000000,2.535334e-02,NaN


In [5]:
import networkx as nx
G = nx.Graph()
for d in dfsig_pos.to_dict('record')[:500]:
    G.add_edge(d['person_x'], d['person_y'], weight=d['prob_xy_obsexp_log'], **d)
G.order(),G.size()

KeyError: 'person_x'

In [ ]:
from pyvis.network import Network
nt = Network(notebook=True, cdn_resources='in_line')
nt.from_nx(G)
nt.show('tmp.nx.html')

ModuleNotFoundError: No module named 'pyvis'